In [ ]:
! pip install tflite-model-maker

In [ ]:
import pandas as pd

import tensorflow as tf

assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

In [ ]:
spec = model_spec.get('mobilebert_classifier')

In [ ]:
train_data = TextClassifierDataLoader.from_csv(
    filename='data/train.csv',
    text_column='Body',
    label_column='Class',
    model_spec=spec,
    delimiter=',',
    is_training=True)

In [ ]:
test_df = pd.read_csv('data/data.csv')
test_df = test_df.dropna()
test_df = test_df[test_df['Class'] != 'skip']
test_df.to_csv('data/data_1.csv', index=False)

In [ ]:
set(test_df['Class'])

In [ ]:
test_data = TextClassifierDataLoader.from_csv(
    filename='data/data_1.csv',
    text_column='Body',
    label_column='Class',
    model_spec=spec,
    delimiter=',',
    is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=spec)

In [ ]:
model.evaluate(test_data)

In [ ]:
config = configs.QuantizationConfig.create_dynamic_range_quantization(
    optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY])
config._experimental_new_quantizer = True

model.export(export_dir='models/mobilebert/unquantized')
model.export(export_dir='models/mobilebert/quantized', quantization_config=config)